<a href="https://colab.research.google.com/github/MariaMaj59/neirons-learn/blob/main/OpenCV_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://pythonpip.ru/opencv/raspoznavanie-obnaruzhenie-lits-v-opencv

import os             # доступ к файлам
import cv2            # работа с изображением
import numpy as np    # работа с массивами

In [ ]:
# ОПРЕДЕЛЯЕМ ПУТЬ И ЗАГРУЖАЕМ МОДЕЛЬ

# Модели prototxt и caffemodel можно взять в репозитории OpenCV. Файл prototxt содержит текстовое описание сети, а caffemodel — вес. 
# Я разместил их в папке model_data. 

# Читаем оба файла и загружаем модель.

base_dir = os.path.dirname(__file__)
prototxt_path = os.path.join(base_dir + 'model_data/deploy.prototxt')
caffemodel_path = os.path.join(base_dir + 'model_data/weights.caffemodel')

model = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

In [ ]:
# СОЗДАЕМ ПАПКИ

if not os.path.exists('updated_images'):
	os.makedirs('updated_images')             # Папка для выходных файлов

if not os.path.exists('faces'):
	os.makedirs('faces')                      # Папка для вырезанных лиц

In [ ]:
# ЧТЕНИЕ ИЗОБРАЖЕНИЙ

# Перебираем все изображения в папке images. 
# Далее делаем проверку на соответствие расширений файлов, чтобы работать только с файлами .png и .jpg.

for file in os.listdir(base_dir + 'images'):
	file_name, file_extension = os.path.splitext(file)
	if (file_extension in ['.png','.jpg']):
		print("Image path: {}".format(base_dir + 'images/' + file))

In [ ]:
# ОБНАРУЖЕНИЕ ЛИЦ

# Используя cv2.imread, читаем изображение и создаем blob с помощью cv2.dnn.blobFromImage. 
# Далее отправляем этот blob в модель и получаем обнаружения, используя model.forward().

image = cv2.imread(base_dir + 'images/' + file)

(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

model.setInput(blob)
detections = model.forward()

In [ ]:
# Создаём боксы вокруг лиц

# Далее перебираем все лица, обнаруженные на изображении, и извлекаем начальную и конечную точки. 
# Затем извлекаем значение достоверности. Если алгоритм более чем на 50% уверен, что обнаружение является лицом, то вокруг него показываем прямоугольник.

# Create frame around face
for i in range(0, detections.shape[2]):
  box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
  (startX, startY, endX, endY) = box.astype("int")

  confidence = detections[0, 0, i, 2]

  # If confidence > 0.5, show box around face
  if (confidence > 0.5):
    cv2.rectangle(image, (startX, startY), (endX, endY), (255, 255, 255), 2)

cv2.imwrite(base_dir + 'updated_images/' + file, image)
print("Image " + file + " converted successfully")

# Далее, используя cv2.imwrite, сохраняем изображение в папку updated_images с тем же именем файла.

In [ ]:
# ИЗВЛЕКАЕМ ЛИЦО

# Как говорилось выше, перебираем все лица, вычисляем достоверность обнаружения и, если она больше 50%, извлекаем лицо. 
# Обратите внимание на строку frame = image[startY:endY, startX:endX]. Здесь происходит извлечение.

# Identify each face
for i in range(0, detections.shape[2]):
  box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
  (startX, startY, endX, endY) = box.astype("int")

  confidence = detections[0, 0, i, 2]

  # If confidence > 0.5, save it as a separate file
  if (confidence > 0.5):
    count += 1
    frame = image[startY:endY, startX:endX]
    cv2.imwrite(base_dir + 'faces/' + str(i) + '_' + file, frame)

# Далее сохраняем это новое изображение в папку faces. 
# Имена файлов присваиваются следующим образом: если имя исходного изображения было sampleImage.png, имя файла после обработки будет 0_sampleImage.png 
# С каждым лицом увеличиваем счётчик и после полного выполнения выводим его на консоль.